[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/jkanclerz/data-science-workshop-2021/blob/main/50--spark-multi-source/10--mongo.ipynb)



In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz -O spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

--2022-01-14 14:15:11--  https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 228834641 (218M) [application/x-gzip]
Saving to: ‘spark-3.1.2-bin-hadoop3.2.tgz’

spark-3.1.2-bin-had 100%[===================>] 218.23M   286MB/s    in 0.8s    

2022-01-14 14:15:12 (286 MB/s) - ‘spark-3.1.2-bin-hadoop3.2.tgz’ saved [228834641/228834641]



In [2]:
!pip install -q pyspark findspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 14.5 MB/s 


In [3]:
!ls

sample_data  spark-3.1.2-bin-hadoop3.2	spark-3.1.2-bin-hadoop3.2.tgz


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [5]:
!sudo apt install mongodb
!sudo service mongodb start

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb-clients
  mongodb-server mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libstemmer0d libyaml-cpp0.5v5 mongo-tools mongodb mongodb-clients
  mongodb-server mongodb-server-core
0 upgraded, 8 newly installed, 0 to remove and 37 not upgraded.
Need to get 53.1 MB of archives.
After this operation, 215 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpcap0.8 amd64 1.8.1-6ubuntu1.18.04.2 [118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libstemmer0d amd64 0+svn585-1build1 [62.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libyaml-cpp0.5v5 amd64 0.5.2-4ubuntu1 [150 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mongo-tools amd64 3.6.3-0ubunt

In [6]:
pip install pymongo

In [7]:
from pymongo import MongoClient
from pprint import pprint as pp
client = MongoClient('mongodb://localhost:27017')
db = client.ecommerce

In [12]:
res = db.inventory.insert_one(
    {
        "Item": "Ruler",
        "Colors": ["Red","Green","Blue","Clear","Yellow"],
        "Inventory": {
            "OnHand": 47,
            "MinOnHand": 40
        },
        "UnitPrice": 0.89
    }
)

In [13]:
!rm /content/spark-3.1.2-bin-hadoop3.2/jars/mongo*

In [14]:

!ls -la /content/spark-3.1.2-bin-hadoop3.2/jars | grep mongo

In [15]:
!wget https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1-assembly.jar -O /content/spark-3.1.2-bin-hadoop3.2/jars/mongo-spark-connector_2.12-3.0.1.jar
!wget https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.10/mongo-java-driver-3.12.10.jar -O /content/spark-3.1.2-bin-hadoop3.2/jars/mongo-java-driver-3.12.10.jar

--2022-01-14 14:16:49--  https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1-assembly.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788378 (2.7M) [application/java-archive]
Saving to: ‘/content/spark-3.1.2-bin-hadoop3.2/jars/mongo-spark-connector_2.12-3.0.1.jar’

/content/spark-3.1. 100%[===================>]   2.66M  --.-KB/s    in 0.03s   

2022-01-14 14:16:49 (101 MB/s) - ‘/content/spark-3.1.2-bin-hadoop3.2/jars/mongo-spark-connector_2.12-3.0.1.jar’ saved [2788378/2788378]

--2022-01-14 14:16:49--  https://repo1.maven.org/maven2/org/mongodb/mongo-java-driver/3.12.10/mongo-java-driver-3.12.10.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... co

In [17]:
sc.stop()

NameError: ignored

In [18]:
from pyspark.sql import SparkSession

working_directory = 'jars/*'
#config("packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0,org.mondodb:mongo-java-driver-3.12.7").\

spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("local").\
        config("spark.mongodb.input.uri","mongodb://localhost:27017/ecommerce.inventory").\
        config("spark.mongodb.output.uri","mongodb://localhost:27017/ecommerce.inventory").\
        getOrCreate()

sc = spark.sparkContext

In [19]:
spark

In [112]:
#sc.stop()

In [20]:
next(db.inventory.find())

{'Colors': ['Red', 'Green', 'Blue', 'Clear', 'Yellow'],
 'Inventory': {'MinOnHand': 40, 'OnHand': 47},
 'Item': 'Ruler',
 'UnitPrice': 0.89,
 '_id': ObjectId('61e185ba8ee9d403e22815d5')}

In [22]:
df = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/ecommerce.inventory").load()

In [24]:

df.show()

+--------------------+---------+-----+---------+--------------------+
|              Colors|Inventory| Item|UnitPrice|                 _id|
+--------------------+---------+-----+---------+--------------------+
|[Red, Green, Blue...| {47, 40}|Ruler|     0.89|{61e185ba8ee9d403...|
|[Red, Green, Blue...| {47, 40}|Ruler|     0.89|{61e185cc8ee9d403...|
+--------------------+---------+-----+---------+--------------------+



In [25]:
sc.stop()